# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(2)
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 累積虧損達股本1/2！中視、華視換照卡關　NCC要求負責人說明

----------------------------------------------------------------------

[1] Disney +推出三個月擁2860萬訂戶　對手Apple TV+壓力倍增

▲Disney+推出不久已累積大量訂閱數。（圖／翻攝自Disney+官網）
記者陳心怡／外電報導
據《The Verge》報導，迪士尼執行長艾格（Bob Iger）在2020會計年度第一季財報電話會議上表示，在Disney+發行後三個月以來，已累積了2860萬訂閱者，遠超過分析師認為的到2020年底有2000萬訂閱戶的預測。
根據艾格說法，Disney+已超越該公司的「最佳期望」。訂閱增長也幫助迪士尼贏得亮眼營收，實現高於預期的286億美元收入。迪士尼的消費者部門在本季增長了9億美元，達到40億美元。迪士尼財務長麥卡錫（Christine McCarthy）指出，迪士尼預計未來的增長將來自國際用戶。
Disney+會擁有大量訂閱數的原因之一，就是有折扣促銷活動，此活動將該服務的訂閱費用降低至3.99美元，並為Verizon訂閱戶提供了免費的Disney+。
此外，Disney+用戶總數已超過了HBO Now用戶數量。迪士尼還因其另外兩項串流媒體服務Hulu和ESPN +而獲得了更多的訂閱量。Hulu的訂閱數從2018年12月底的2280萬，增長至2019年底的3040萬。ESPN+則從2018年12月底的140萬，增至2019年底的660萬。
此前，蘋果於上周發布的財報中，並沒有特別提到Apple TV+的營收。蘋果執行長庫克（Tim Cook）告訴投資者，該公司對Apple TV+的強勁開局感到滿意，但此數字似乎與迪士尼的新數字無法比擬。
伯恩斯坦研究公司（Bernstein Research）分析師薩科納吉（Toni Sacconaghi）從蘋果第一季財報中推斷，最近Apple TV+的採用率很低，即使在那些能免費使用它的國家中也是如此。他推測這可能是由於Apple TV+上的內容數量有限，或是蘋果的宣傳仍無法成功吸引人們

▲中經院院長陳思寬。（圖／記者余弦妙攝）
記者余弦妙／台北報導
中華經濟研究院今（5）日公布，1月台灣非製造業經理人指數（NMI）已連續11個月呈現擴張，且指數微升0.9個百分點至55.0%。四項組成指標中，商業活動、新增訂單與人力僱用呈現擴張，供應商交貨時間呈現上升。
中經院表示，原本中美貿易紛爭對產業的影響已大減，但是因為之後出現武疫情，且開學延後導致暑假延後等，已有旅遊業、零售業及航運業接到訂單取消，製造業消費可以回補，但非製造業訂單不能回補，因此非製造業未來不確定性比製造業更高，較令人憂心，預估下個月採購經理人指數會劇烈波動。
另外，1月非製造業NMI也是連續11個月擴張，受到製造業好轉而動能慢慢往上，指數微升0.9，指數達到55。四項指標中，商業活動、新增訂單及人力僱用都呈現擴張，供應商交貨也上升。各產業擴張速鹿依序是，住宿餐飲業（75%）、資訊暨通訊傳播業（58.7%）、教育及專業科學業（56.5%）、營造暨不動產業（55.8%）、運輸倉儲業（55.8%）、金融保險業（53.5%）、批發業（51.3%），其中只有零售業較上月呈現緊縮。
中經院院長陳思寬指出，1月的調查時間是在武漢肺炎疫情擴大前，PMI、NMI兩個指數都續呈擴張，但最近疫情延燒，廠商開工日期延後，不確定因素增加。
陳思寬也提到，從之前經驗看起來的話，這是一個比較短時間的衝擊，製造業等到過去之後，這些需求會回補，所以是還好，但非製造業像是旅遊啦，時間過去了就過去了，恢復狀況相對會比較緩慢一些。
----------------------------------------------------------------------

[14] 酒精也缺貨！台糖今召集製藥業者「代工新增百萬瓶產能」

▲武漢肺炎疫情持續延燒，不僅口罩一罩難求，防疫產品如消毒酒精也搶翻天，造成市面上大缺貨。（圖／記者周宸亘攝）
記者林淑慧／台北報導
為因應武漢肺炎防疫需求，政府統計國內消毒酒精仍有40萬瓶需求缺口，台糖今（5）日表示，目前已開三班制趕工中，產能已較往前提升五成，也將召集民間製藥業者洽談代工，預計新增百萬瓶產能，以滿足國內防疫消毒需求。
武漢肺炎疫情持續延燒，不僅口罩一罩難求，防疫產品如消毒酒精搶翻天，大批民眾至藥局、美妝店排隊購買酒精。台糖表示，民眾除了戴口罩避免感染外，勤洗手及消毒亦是阻擋新型

4日傳出富士康等工廠因中國各地政府規定，必需延至10日開工，可能波及蘋果供應鏈，導致iPhone出不了貨；然而美國「DA Davidson」資深分析師福特（Tom Forte）指出，只要情況在2月中前解決，蘋果就不太會受到疫情影響。
事實上，蘋果反而大漲3.30％，帶動美股4日的升勢，臉書也上漲2.76％、Netflix漲3.08％、亞馬遜也托2.27％漲勢之福，市值重新上兆。
另一方面，特斯拉4日股價一度飆到每股969美元的高位，短暫超越Netflix，終場上漲13.73％，過去2日更漲幅多達36％。
----------------------------------------------------------------------

[27] 延後開學嚇壞家長　台達電DeltaMOOCx線上數理免費課程照常上

▲台達電DeltaMOOCx線上數理免費課程。（圖／業者提供）
記者周康玉／台北報導
受到新型冠狀病毒疫情影響，教育部已宣佈全國高中職以下學生延後開學，寒假輔導課也取消，許多家長擔心孩子的學習因此受到影響。由台達電子文教基金會開設的免費線上課程平台「DeltaMOOCx」則不受影響，即使延後開學，照樣可使用。
DeltaMOOCx可以讓高中、高工的學生透過網路線上學習數理相關課程，達到預習和複習的雙重效果。除平台外，所有影片也都可以免費在YouTube上搜尋到。只要輸入關鍵字DeltaMOOCx，即可找到一系列的高中數學、物理、化學、生物等課程，老師們也可以透過連結DeltaMOOCx的影片，遠距與自己的班級學生進行「線上寒假輔導」，讓學生們不至因疫情荒廢學業。
DeltaMOOCx計畫主持人清華大學講座教授表彭宗平示，在疫情嚴峻的當下，線上課程正可以讓老師與學生多加利用，讓學習跳脫時空的限制。不論是利用手機、平板、或電腦等各種載具，都可以觀看影片與練習題目；學生們也可以在平台上發問，老師們也會在線上回答問題，相信可以減緩此波疫情對於學習進度的影響。
DeltaMOOCx是由台達基金會、國家教育研究院以及數學、物理、電機電子等學群科中心所共同合作，由各高中、高工一線優秀老師所拍攝，委託愛爾達電視台製作。自2015年起，課程已陸續於「DeltaMOOCx愛學網」及YouTube「DeltaMOOCx頻道」上線。現已有162所高中職與12所大專院校加

▲新型冠狀病毒疫情嚴重，全球銅價不斷下跌。（圖／路透社）
記者謝仁傑／綜合報導
銅的價格可以說是全球經濟健康的指標，據CNBC報導，自1月中以來已下跌逾12％，因為新型冠狀病毒（武漢肺炎）的爆發迫使中國延長製造業地區的停產期，本週一是銅價連續下跌的第13天，跌至去年9月以來的低點。
銅被廣泛用於製造，包括電子產品到建築住宅。中國是世界上最大的金屬消費國。
Charles Schwab的首席策略師Jeffrey Kleintop表示：「銅與製造業緊密地聯繫在一起。製造業是全球經濟最薄弱的第一線，令人擔心的是，這是繼去年貿易衝擊後，全球經濟和製造業的二度衝擊，這足以讓銅與其他金屬價格持續疲乏。」
週一，上海銅價也跌至近三年低點，上海期貨交易所的銅交易更是下跌7％。上海期貨交易所已宣告暫停夜間交易。
澳新銀行（ANZ Research）上周在一份報告中指出，中國主要產區的停產將進一步衝擊銅價，而中國許多城市也嚴格實施了旅行限制。
----------------------------------------------------------------------

[41] 【廣編】不用中樂透！興富發建設「早鳥預約」9折限量回饋中

▲興富發大方祭出40週年感恩回饋優惠方案，上網預約早鳥9折買房大優惠。（圖／興富發建設提供，下同）
圖、文／興富發建設
不用中樂透  「早鳥預約」開春大紅包
早鳥可以預約買什麼，預約買高鐵票、買演唱會的票，預約尾牙餐廳、預約年節禮盒、預約國外高檔旅行團，但這些回饋都抵擋不了，金鼠年最強的回饋：「上網預約買房，享9折優惠。」網友驚呼：「省下了錢，讓你鼠錢鼠不完啊！」也有網友說：「已預約啦，還沒預約的是在哈囉？」
破天荒！上網預約早鳥9折限量回饋中 
全台建商龍頭興富發集團欣逢40週年，日前慶祝獲得第4屆爆米花棒球聯盟冠軍，興富發總裁鄭欽天在集團尾牙時，大方祭出40週年感恩回饋優惠方案，上網預約早鳥9折買房大優惠，讓全台灣購屋族湧入網站，預約盛況空前。
凡即日起至2020/02/29（六），前往興富發官方賞屋平台「早鳥專區」完成預約登記，享興富發子公司巨豐管理代銷之北中南五大全新個案9折優惠，包括台中七期案、桃園青溪案、台南漁光案、高雄農十六案、高雄民生臻愛2，簽約完成加碼再贈時下最夯「電動自行車一台」。
大手筆回饋就是了解新一代購屋族的辛

▲ 台股全面逐筆交易與盤中零股交易制度將於今年實施。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股逐筆交易將於3月23日上線，再6個月後也將實施盤中零股交易。券商公會理事長賀鳴珩認為，這兩項是台灣證券市場必要的基礎工程，零股交易更是金融創新關鍵。
賀鳴珩指出，目前電子下單及全面化作業已經普及，實施盤中零股交易的時機已經成熟，將在逐筆上線後的6個月實施，但需要搭配幾個前提，提升交易便利，零股交易才會成功。證交所也已在去年底公告新增額度100萬以内的新客戶，可手持身分證件拍照搭配證券商分戶帳指定出金帳戶辦理網路開戶。
▲ 券商公會理事長賀鳴珩。（圖／記者湯興漢攝）
此外，亦推廣開立分戶帳，讓投資人可以在證券商同時完成開立證券戶與分戶帳帳戶，不用再到銀行開交割專戶。並持續鼓勵從開戶、下單、帳務到集保都要全面e化，才能降低成本。
賀鳴珩認為，撮合間隔時間縮短才能滿足投資人交易習慣、滅少交易糾紛，他表示，零股交易希望能將3分鐘撮合一次改為5秒鐘撮合一次，但也要待逐筆交易上線且穩定了之後，在來研議是否有可能將零股交易撮合時間縮短。
券商公會表示，為利發展金融科技及普惠金融，將持續研議優化盤中零股交易相關配套措施，如：交易撮合頻率、引進造市機制、強化投資人下單系統辨識、研議電子憑證收費方案等，以積極吸引年輕族群及小資族參與，對開發新客群、證券業AI、產品創新等目標發揮關鍵影響，亦可充分與國際制度平行接軌。
另為讓證券從業人員及投資人瞭解盤中零股交易與現行盤後零股交易不同，券商公會將與證交所、櫃買中心共同加強從業人員的教育訓練，以利第一線的從業人員與投資人順利執行盤中零股交易。
----------------------------------------------------------------------

[54] 兩廳院售票系統升級！選位可「視野預覽」、AI智慧學習觀眾喜好

記者林育綾／台北報導
耗時一年多的研發，「兩廳院售票系統」將全面「打掉重練」，升級為「OpenTix」文化生活電商平台（暫名），不僅選位更便捷，還有「視野預覽」功能，加上更具智慧的全文檢索，方便觀眾找尋節目，還有AI 智慧學習模式，能瞭解觀眾對表演和選位的喜好，未來自動推薦。全新的系統預計4月至5月左右開放公測，期望能在今年第四季上線！
▲升級的票務系統，未來有全文檢索功能，找尋喜愛的節目更

開發金今（4）日公告，開發金總經理、中壽董事長董事長王銘陽退休，其法人董事職缺待基捷投資股份有限公司另行指派，並經由董事會選出新任總經理接替。根據了解，王銘陽雖然外表十分年輕，今年仍未滿64歲，而且目前開發金日前才向金管會申請延長併中壽的時程、以及及中壽、開發金的新總部大樓即將落成，選擇此時退休引發外界好奇。
曾經在南山人壽服務，從基層的投資部做起，歷經主任、經理、副總經理等職務，在壽險業相當受矚目的菁英，也在中壽任總經理、在開發金購買中壽35%股權，轉任開發金總經理、中壽董事長。其學歷為美國德州大學達拉斯分校管理科學研究所碩士。
根據開發金的公告，目前尚未決定接任人選，開發金表示，總經理的職缺要等待基捷投資另行指派；中壽的公告則是，要等待開發金決定。
----------------------------------------------------------------------

[65] 外傳口罩原料短缺？經部：原料供應無虞、新產線三月起陸續上線

▲武漢肺炎疫情持續延燒，造成全球口罩搶購熱潮。（圖／路透）
記者林淑慧／台北報導
武漢肺炎疫情持續升溫，全球瘋搶口罩，外傳台灣口罩的不織布原料恐將短缺；對此，經濟部長沈榮津召集口罩業者與會，業者大多表示原料供應無虞，將力拚新產線於三月起陸續上線，以滿足國內口罩需求的缺口。
經濟部表示，政府對口罩生產的原料一直都有掌握，從PP聚丙烯（不織布原料）、到口罩內層ES、中間層熔噴布、外層紡黏，國內都有生產，經濟部將隨時注意原物料的供應情況。
「（經濟）部長昨天的指令很明確，就是衝高產能、確保物料。所以新產線開始後，部長請林次（經濟部常務次長林全能）盤點後續的原物料供應無虞，他也會再找業者討論。」經濟部官員指出。
依據經濟部最新統計，目前國內口罩日產能為400萬片，官員指出，目前已經有不少製造商，採取全天候輪班生產的方式製造口罩，而本月政府斥資2億佈建的 62條口罩產線也將陸續上線，國內產能可望逐步提高。
----------------------------------------------------------------------

[66] 和泰產險董事長由「前全球人壽總經理」張永固出任　劉源森續任副董事長

▲和泰產物保險改選董事。（圖／《ETtoday新聞雲》資料照）
記者楊絡懸／台北報導
和泰

▲大樂透今天晚上頭獎上看3億元，剩餘99組春節大紅包將於今天、周五全數開出。（圖／記者許力方攝）
記者紀佳妘／台北報導
台彩今年農曆春節電腦型彩券加碼總獎金8億元，大樂透從小年夜起連開11天，還有360組春節大紅包，僅初一、初三開出頭獎外，其餘都槓龜、已連續6期槓龜，因此大樂透今（4）日頭獎上看3億元，剩餘99組春節大紅包100萬將於今天、周五全數開完。
台彩總經理蔡國基表示，今年春節期間大樂透開出2次頭獎，較過往的3、4期來得少。大樂透今晚頭獎上看3億元，還有99組春節百萬大紅包，民眾購買一張大樂透，不僅可以對億元頭獎，還有機會對中春節大紅包獎項，等於是買一張彩券可以對2種獎項。
今年春節有7天連假，大樂透一共開出2次頭獎，其中一次是初一，當時頭獎為2.4億元，一注獨得，獎落在台東市豐田里的「鑫富發彩券行」；另一次則是在初三，頭獎1.2億元一注獨得，獎落在新北市林口區的「百勝投注站」。
蔡國基指出，刮刮樂今年100萬元（含）以上的獎項有646個，到昨天為止，100萬元（含）以上獎項已開出401個，換句話說，還有245個100萬元（含）以上獎項待開出，其中最受民眾喜愛的是「2000萬超級紅包」刮刮樂，頭獎2,000萬元共有3個，目前已被刮出一個，獎落在桃園中壢的「伍拾壹號彩券行」，這次幸運兒是一名年約40多歲的女性上班族，幸運刮中2,000萬元，其中捐贈8.8萬元給中信慈善基金會。
不過，今年春節期間電腦型彩券加刮刮樂日均銷售量為23億元，較去年春節期間的日均銷售22.87億元，小幅成長0.4%。蔡國基說，原先預估日均銷售量29億元，至少有20%成長，但受到武漢肺炎疫情和春節天數影響，並不如預期。
至於威力彩部分，蔡國基指出，從1月20日加碼2億後，頭獎已連續9期槓龜，6日頭獎上看4.5億，也是今年來最高的頭獎，民眾不妨把握高額頭獎的機會。
----------------------------------------------------------------------

[76] TCL宣布8月31日起停售黑莓機！　現有設備支援保修至2022年8月31日

▲經典黑莓機。（圖／ETtoday資料照）
記者陳心怡／外電報導
經典的黑莓機是許多人的回憶，不過BlackBerry Mobile發布最新消息，自2020年8月31日起，現在持有和營運黑莓手機品牌的

▲台銀總行。（圖／記者吳靜君攝）
記者吳靜君／台北報導
新春開工，台銀推出「小微企業優利簡易貸款專案」，總額度50億元，每戶額度最高50萬元，由中小企業信用保證基金保證，免提供擔保品，總利費率僅1.98%，預期將有1萬戶小微企業受惠。
台銀為支持政府推動普惠金融政策，強化中小企業競爭力與微型企業轉型、升級等之資金需求，特別提撥自有資金50億元辦理「小微企業優利簡易貸款專案」。
今年新春開工後適值武漢肺炎國際疫情升溫期間，台銀為全力支持小微企業共體時艱齊心抗疫外，也提供優惠貸款專案做為抗疫衍生所需營運週轉金。
台銀專案對象涵蓋有稅籍登記德小微企業，及無稅籍登記且有營業事實之小農、小漁等自然人，並符合中小企業信用保證基金「中小企業千億保專案」的貸款對象者，供其營運週轉之需，免提供擔保品，每戶額度最高50萬元，期間最長三年，借款人負擔總利費率僅1.98%，預期逾1萬戶以上之小微企業受惠。
台銀表示，該行支持中小企業發展與提供融資協助成長一向不遺餘力，截至去（2019）年底止，對中小企業放款總戶數達19,026戶，較2018年底大幅增加4,178戶，成長幅度28.1%，新承作額度淨增加623億元，顯見對於中小企業的支持。
----------------------------------------------------------------------

[87] 鴻海富士康10日若無法開工　路透：iPhone恐有斷貨危機

▲鴻海旗下富士康龍華廠區。（圖／翻攝富士康）
記者姚惠茹／綜合報導
武漢肺炎疫情擴散，鴻海（2317）旗下富士康配合中國大陸官方防疫政策延後至2月10日復工，現在有消息指出，富士康如果停產持續到第2週，無法順利在10日復工的話，不僅將面臨巨大的生產影響，甚至會衝擊包括蘋果在內的客戶面臨斷貨危機。
根據《路透》引述消息人士報導，富士康為蘋果和其它品牌生產智慧型手機，但是受到武漢肺炎爆發影響，富士康配合中國大陸官方防疫政策延後至2月10日復工，已經停止在中國大陸幾乎全部的生產工作，其中就包括蘋果在內的客戶產品。
消息人士指出，富士康利用越南、印度、墨西哥等地的工廠來填補中國大陸工廠停工的缺口，因此到目前為止對富士康的衝擊不大，而且富士康目前期望停工令解禁後，透過工廠員工加班的方式彌補產能的不足。
目前富士康復工時間取決於武漢肺炎的疫情狀況與中國大陸